In [ ]:
from aeon.datasets.tsc_datasets import univariate, multivariate

print("Univariate datasets:", univariate)
print("Multivariate datasets:", multivariate)

In [ ]:
from aeon.datasets import load_classification
X, y, meta_data = load_classification("GunPoint", return_metadata=True)
print(" Shape of X = ", X.shape)
print(" Meta data = ", meta_data)

In [ ]:
import numpy as np
import pandas as pd

records = []
for idx in range(X.shape[0]):
    for t in range(X.shape[2]):
        records.append({
            "series": idx,
            "time_idx": t,
            "value": X[idx, 0, t],
            "label": y[idx]
        })
df = pd.DataFrame(records)

In [ ]:
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
import time
start = time.time()
max_encoder_length = 50
max_prediction_length = 10

dataset = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="value",
    group_ids=["series"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["label"],
    target_normalizer=GroupNormalizer(groups=["series"])
)
end = time.time()
print(f"Dataset loading time: {end - start:.4f} seconds")


In [ ]:
start = time.time()
dataloader = dataset.to_dataloader(batch_size=64)
batch = next(iter(dataloader))
end = time.time()
print(f"Preprocessing and batch extraction time: {end - start:.4f} seconds")

In [ ]:
import psutil
import os

process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1e6
dataset = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="value",
    group_ids=["series"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    static_categoricals=["label"],    target_normalizer=GroupNormalizer(groups=["series"])
)
mem_after = process.memory_info().rss / 1e6
print(f"Memory used by dataset: {mem_after - mem_before:.2f} MB")


In [ ]:
# scaling
from pytorch_forecasting import TimeSeriesDataSet
from sklearn.preprocessing import StandardScaler

scalers = {
    "my_continuous_feature": StandardScaler()
}

dataset = TimeSeriesDataSet(
    data=df,
    time_idx="time_idx",
    target="target",
    group_ids=["series"],
    max_encoder_length=24,
    max_prediction_length=6,
    time_varying_unknown_reals=["my_continuous_feature"],
    scalers=scalers
)

In [ ]:
import time
import psutil
import os
from torch.utils.data import DataLoader

process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1e6

start = time.time()
dataset = TimeSeriesDataSet(
    data=df,
    time_idx="time_idx",
    target="target",
    group_ids=["series"],
    max_encoder_length=30,
    max_prediction_length=7,
    time_varying_unknown_reals=["target"],
    randomize_length=(0.8, 1.0)
)
creation_time = time.time() - start

dataloader = dataset.to_dataloader(batch_size=64)
start = time.time()
for batch in dataloader:
    pass
iteration_time = time.time() - start

mem_after = process.memory_info().rss / 1e6

print(f"Dataset creation time: {creation_time:.2f}s")
print(f"DataLoader iteration time: {iteration_time:.2f}s")
print(f"Memory used: {mem_after - mem_before:.2f}MB")

In [ ]:
import time
import psutil
import os

start = time.time()
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1e6

dataset = TimeSeriesDataSet(
    data=df,
    time_idx="time_idx",
    target="target",
    group_ids=["series"],
    max_encoder_length=2,
    max_prediction_length=1,
    time_varying_known_reals=["feature1", "feature2"],
    allow_missing_timesteps=True,
    constant_fill_strategy={
        "feature1": "ffill",
        "feature2": "bfill",
        "target": 0
    }
)
creation_time = time.time() - start
mem_after = process.memory_info().rss / 1e6

print(f"Dataset creation time: {creation_time:.4f}s")
print(f"Memory used: {mem_after - mem_before:.2f}MB")
